Here's the basic way to connect to postgres with psycopg2.

use postgresondocker  at the prompt for db, username, and password to begin with. 

In [3]:
import psycopg2
import getpass

# logging into db

def db_login_and_conn():
    db=input('enter dbname: ')
    user=input('enter user: ')
    print('enter password: ')
    password=getpass.getpass()
    conn_str="host='localhost' dbname='"+db+"' user='"+user+"' password='"+password+"' \
    options='-c search_path=sandbox,public'"
    # the options allows us to add a schema called sandbox and be able to work in it.
    # because sandbox is first, tables will be created there if schema isn't specified.
    conn=psycopg2.connect(conn_str)
    return conn
    

In [4]:
print('\nuse postgresondocker at prompt for db,username,password\n')
conn = db_login_and_conn() # get database, username, password

cur=conn.cursor()
cur.execute('select version()')
print(cur.fetchone()[0])


use postgresondocker at prompt for db,username,password



enter dbname:  postgresondocker
enter user:  postgresondocker


enter password: 


 ················


PostgreSQL 9.3.17 on x86_64-unknown-linux-gnu, compiled by gcc (Ubuntu/Linaro 4.6.3-1ubuntu5) 4.6.3, 64-bit


In [5]:
# if code errors out, uncomment next line and run a commit so you can keep running code
# conn.commit()

In [6]:
str_sql = "SELECT schema_name FROM information_schema.schemata;"
cur.execute(str_sql)
conn.commit()
schemas =cur.fetchall()
print('list of all schemas before attempting to add sandbox:')
print(schemas)

list of all schemas before attempting to add sandbox:
[('pg_toast',), ('pg_temp_1',), ('pg_toast_temp_1',), ('pg_catalog',), ('public',), ('information_schema',), ('sandbox',)]


In [7]:

str_sql='CREATE SCHEMA IF NOT EXISTS sandbox;'
cur.execute(str_sql)
conn.commit()

str_sql = "SELECT schema_name FROM information_schema.schemata;"
cur.execute(str_sql)
conn.commit()
schemas =cur.fetchall()
print('list of all schemas after adding sandbox:')
print(schemas)


list of all schemas after adding sandbox:
[('pg_toast',), ('pg_temp_1',), ('pg_toast_temp_1',), ('pg_catalog',), ('public',), ('information_schema',), ('sandbox',)]


In [8]:

str_sql = "SELECT table_name FROM information_schema.tables where table_schema='sandbox' AND table_type='BASE TABLE';"
cur.execute(str_sql)
conn.commit()
schemas =cur.fetchall()
print('list of all tables in schema sandbox before adding test:')
print(schemas)


str_sql='CREATE TABLE IF NOT EXISTS sandbox.test(message varchar(50));'
cur.execute(str_sql)

str_sql = "SELECT table_name FROM information_schema.tables WHERE table_schema='sandbox' AND table_type='BASE TABLE';"
cur.execute(str_sql)
conn.commit()
schemas =cur.fetchall()
print('list of all tables in schema sandbox after adding test:')
print(schemas)



list of all tables in schema sandbox before adding test:
[]
list of all tables in schema sandbox after adding test:
[('test',)]


In [9]:

str_sql="INSERT INTO sandbox.test(message) VALUES('bk was here');"
cur.execute(str_sql)
str_sql="INSERT INTO sandbox.test(message) VALUES('you can be too!');"
cur.execute(str_sql)

str_sql='SELECT * FROM sandbox.test;'
cur.execute(str_sql)
print('data in table test')
print(cur.fetchall())
print(cur.fetchall())
conn.commit()


data in table test
[('bk was here',), ('you can be too!',)]
[]


In [10]:

str_sql="DROP TABLE sandbox.test;"
cur.execute(str_sql)
conn.commit()


str_sql = "SELECT table_name FROM information_schema.tables WHERE table_schema='sandbox' AND table_type='BASE TABLE';"
cur.execute(str_sql)
conn.commit()
schemas =cur.fetchall()
print('list of all tables after dropping test:')
print(schemas)


list of all tables after dropping test:
[]


In [11]:
# get a list of databases
str_sql='select datname from pg_database;'
cur.execute(str_sql)
print(cur.fetchall())
conn.commit()


[('template1',), ('template0',), ('postgres',), ('postgresondocker',), ('flit',)]


In [12]:
# create a new database
conn.autocommit=True # create db can't run in a transaction
cur = conn.cursor()
str_sql='CREATE DATABASE testdb;'
cur.execute(str_sql)

str_sql='select datname from pg_database;'
cur.execute(str_sql)
print(cur.fetchall())
conn.commit()


[('template1',), ('template0',), ('postgres',), ('postgresondocker',), ('flit',), ('testdb',)]


In [13]:
str_sql='DROP DATABASE testdb;'
cur.execute(str_sql)

str_sql='select datname from pg_database;'
cur.execute(str_sql)

print(cur.fetchall())

[('template1',), ('template0',), ('postgres',), ('postgresondocker',), ('flit',)]


In [22]:
str_sql="CREATE USER flit WITH PASSWORD 'flit' CREATEDB;"
cur.execute(str_sql)
str_sql='CREATE DATABASE flit;'
cur.execute(str_sql)
str_sql='GRANT ALL ON DATABASE flit TO flit;'
cur.execute(str_sql)


print('users')
str_sql='select usename from pg_user;'
cur.execute(str_sql)
print(cur.fetchall())


users
[('postgres',), ('postgresondocker',), ('bk',), ('flit',)]


We have a new user with createdb permission, we can close and reopen the connection with the new user name.

In [23]:
conn = db_login_and_conn() # get database, username, password
cur=conn.cursor()
cur.execute('select version()')
print(cur.fetchone()[0])

enter dbname:  flit
enter user:  flit


enter password: 


 ····


PostgreSQL 9.3.17 on x86_64-unknown-linux-gnu, compiled by gcc (Ubuntu/Linaro 4.6.3-1ubuntu5) 4.6.3, 64-bit


In [24]:
str_sql='CREATE SCHEMA IF NOT EXISTS sandbox;'
cur.execute(str_sql)
str_sql='CREATE TABLE IF NOT EXISTS sandbox.test(message varchar(50));'
cur.execute(str_sql)
str_sql="INSERT INTO sandbox.test(message) VALUES('flit was here');"
cur.execute(str_sql)
str_sql="INSERT INTO sandbox.test(message) VALUES('you can be too!');"
cur.execute(str_sql)
str_sql='SELECT * FROM sandbox.test;'
cur.execute(str_sql)
print('data in table test')
print(cur.fetchall())
conn.commit()


data in table test
[('flit was here',), ('you can be too!',)]


In [25]:
conn.close()